In [1]:
import calendar
import os
import time

import skyfield
from tables import *

import zipfile
from skyfield.sgp4lib import EarthSatellite

In [2]:
class TLE(IsDescription):
    epoch     = Float64Col(pos=0)
    norad_id  = Int64Col(pos=1)
    line_one  = StringCol(80, pos=2)
    line_two  = StringCol(80, pos=3)

export_path = "/mnt/disk100/persist_home/meawoppl/reexport.h5"
h5file = open_file(export_path, mode="w", title="TLE Indexable Data")
table = h5file.create_table(h5file.root, 'tle', TLE, "Main TLE Listing")

In [3]:
table

/tle (Table(0,)) 'Main TLE Listing'
  description := {
  "epoch": Float64Col(shape=(), dflt=0.0, pos=0),
  "norad_id": Int64Col(shape=(), dflt=0, pos=1),
  "line_one": StringCol(itemsize=80, shape=(), dflt=b'', pos=2),
  "line_two": StringCol(itemsize=80, shape=(), dflt=b'', pos=3)}
  byteorder := 'little'
  chunkshape := (372,)

In [4]:
h5file

File(filename=/mnt/disk100/persist_home/meawoppl/reexport.h5, title='TLE Indexable Data', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'TLE Indexable Data'
/tle (Table(0,)) 'Main TLE Listing'
  description := {
  "epoch": Float64Col(shape=(), dflt=0.0, pos=0),
  "norad_id": Int64Col(shape=(), dflt=0, pos=1),
  "line_one": StringCol(itemsize=80, shape=(), dflt=b'', pos=2),
  "line_two": StringCol(itemsize=80, shape=(), dflt=b'', pos=3)}
  byteorder := 'little'
  chunkshape := (372,)

In [5]:
tle_folder = "/mnt/disk100/data/TLE"
zips = [os.path.join(tle_folder, f) for f in os.listdir(tle_folder)]

In [6]:
def fmt_epoch(epoch_val):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(epoch_val))


In [7]:
entry = table.row

def read_tles_from_zip(path: str) -> int:
    print("Reading:" + path)
    z = zipfile.ZipFile(path)
    name = list(filter(lambda fn: "_MACOSX" not in fn, z.namelist()))[0]
    print(name)
    size = 0

    with zipfile.ZipFile(path) as z:
        with z.open(name) as f:
            while True:
                tle1 = f.readline().decode().strip()
                if not tle1: break
                tle2 = f.readline().decode().strip()
                sat = EarthSatellite(tle1, tle2)
                entry["epoch"] = sat.epoch.utc_datetime().timestamp()
                entry["norad_id"] = sat.model.satnum
                entry["line_one"] = tle1
                entry["line_two"] = tle2
                entry.append()
            
                size += 1
                if (size % 100000 == 0):
                    table.flush()
                    print("Exported %i rows to %s" % (table.shape[0], fmt_epoch(table[-1][0])))
    table.flush()
    print(table[-1][0])
    print("Exported up to %s" % fmt_epoch(table[-1][0]))

In [8]:
for tlez in sorted(zips):
    read_tles_from_zip(tlez)

print("Building indices")
h5file.root.tle.cols.epoch.create_index()
h5file.root.tle.cols.norad_id.create_index()
    
print("Done..")
h5file.close()

Reading:/mnt/disk100/data/TLE/tle2004_1of8.txt.zip
tle2004_1of8.txt
Exported 100000 rows to 2001-05-05 20:03:47
Exported 200000 rows to 2001-05-19 18:03:22
Exported 300000 rows to 2001-06-01 19:17:33
Exported 400000 rows to 2001-06-15 22:23:59
Exported 500000 rows to 2001-06-29 13:36:44
Exported 600000 rows to 2001-07-13 11:07:53
Exported 700000 rows to 2001-07-27 22:19:32
Exported 800000 rows to 2001-08-10 15:54:08
Exported 900000 rows to 2001-08-24 18:46:14
Exported 1000000 rows to 2001-09-08 19:10:15
Exported 1100000 rows to 2001-09-20 16:31:53
Exported 1200000 rows to 2001-10-06 15:40:28
Exported 1300000 rows to 2001-10-20 15:11:19
Exported 1400000 rows to 2001-11-04 17:14:36
Exported 1500000 rows to 2001-11-18 00:01:19
Exported 1600000 rows to 2001-12-02 05:22:26
Exported 1700000 rows to 2001-12-17 21:57:25
Exported 1800000 rows to 2001-04-30 23:17:50
1085716314.746765
Exported up to 2004-05-28 03:51:54
Reading:/mnt/disk100/data/TLE/tle2004_2of8.txt.zip
tle2004_2of8.txt
Exported 1

Exported 16681402 rows to 1989-01-31 22:46:25
Exported 16781402 rows to 1989-10-23 01:16:42
Exported 16881402 rows to 1989-05-11 09:20:47
Exported 16981402 rows to 1989-06-03 04:18:02
Exported 17081402 rows to 1989-12-31 14:55:04
Exported 17181402 rows to 1990-03-11 01:50:09
Exported 17281402 rows to 1990-03-26 15:58:04
Exported 17381402 rows to 1990-05-09 12:41:28
Exported 17481402 rows to 1990-11-25 03:09:23
Exported 17581402 rows to 1990-01-13 05:58:38
Exported 17681402 rows to 1990-11-25 05:28:58
Exported 17781402 rows to 1990-11-30 15:07:06
Exported 17881402 rows to 1990-05-28 06:06:57
Exported 17981402 rows to 1991-10-26 13:28:00
Exported 18081402 rows to 1991-03-25 00:28:39
Exported 18181402 rows to 1991-08-23 18:13:30
Exported 18281402 rows to 1991-07-23 07:58:22
Exported 18381402 rows to 1991-09-02 13:19:36
Exported 18481402 rows to 1991-07-03 01:16:00
Exported 18581402 rows to 1992-03-12 00:23:14
Exported 18681402 rows to 1992-01-02 06:45:55
Exported 18781402 rows to 1992-01-

Exported 34581402 rows to 2000-09-28 04:19:09
Exported 34681402 rows to 2000-10-07 20:56:55
Exported 34781402 rows to 2000-10-20 02:14:30
Exported 34881402 rows to 2000-10-27 21:54:27
Exported 34981402 rows to 2000-11-11 00:33:08
Exported 35081402 rows to 2000-11-21 18:13:59
Exported 35181402 rows to 2000-12-01 21:47:47
Exported 35281402 rows to 2000-12-12 19:57:58
Exported 35381402 rows to 2000-12-25 08:11:33
Exported 35481402 rows to 2001-01-02 11:08:03
Exported 35581402 rows to 2001-01-13 12:36:29
Exported 35681402 rows to 2001-01-22 23:06:04
Exported 35781402 rows to 2001-02-05 01:21:47
Exported 35881402 rows to 2001-02-14 22:10:58
Exported 35981402 rows to 2001-02-25 16:44:34
Exported 36081402 rows to 2001-03-08 17:17:02
Exported 36181402 rows to 2001-03-20 12:18:49
Exported 36281402 rows to 2001-03-29 22:54:18
Exported 36381402 rows to 2001-04-10 03:33:22
Exported 36481402 rows to 2001-04-22 08:33:36
Exported 36581402 rows to 2001-05-03 15:20:41
Exported 36681402 rows to 2001-06-

Exported 52075817 rows to 1994-03-04 00:58:49
Exported 52175817 rows to 1994-02-09 20:54:30
Exported 52275817 rows to 1994-03-22 01:16:07
Exported 52375817 rows to 1994-01-08 22:44:03
Exported 52475817 rows to 1994-08-13 07:16:01
Exported 52575817 rows to 1994-02-12 03:26:26
Exported 52675817 rows to 1994-06-04 01:58:30
Exported 52775817 rows to 1994-01-26 05:06:48
Exported 52875817 rows to 1994-09-25 05:29:17
Exported 52975817 rows to 1994-02-07 07:41:06
Exported 53075817 rows to 1994-08-14 04:56:56
Exported 53175817 rows to 1994-01-24 17:52:19
Exported 53275817 rows to 1994-12-26 02:45:47
Exported 53375817 rows to 1994-02-13 01:39:36
Exported 53475817 rows to 1994-01-25 10:27:39
Exported 53575817 rows to 1994-09-18 05:22:00
Exported 53675817 rows to 1994-07-23 14:51:18
Exported 53775817 rows to 2005-06-26 17:12:07
Exported 53875817 rows to 2005-06-11 22:20:30
Exported 53975817 rows to 2005-06-29 22:41:20
Exported 54075817 rows to 2005-07-12 21:43:02
Exported 54175817 rows to 2005-07-

Exported 69070417 rows to 2008-11-28 11:48:14
Exported 69170417 rows to 2008-12-17 10:35:07
Exported 69270417 rows to 2009-01-03 11:57:06
Exported 69370417 rows to 2009-01-13 09:13:53
Exported 69470417 rows to 2009-01-19 12:08:26
Exported 69570417 rows to 2009-01-27 14:51:39
Exported 69670417 rows to 2009-02-04 12:15:03
Exported 69770417 rows to 2009-02-10 20:13:41
Exported 69870417 rows to 2009-02-18 09:52:05
Exported 69970417 rows to 2009-02-26 09:45:29
Exported 70070417 rows to 2009-03-04 20:23:24
Exported 70170417 rows to 2009-03-11 19:12:26
Exported 70270417 rows to 2009-03-18 16:30:54
Exported 70370417 rows to 2009-03-25 13:14:05
Exported 70470417 rows to 2009-03-31 19:37:51
Exported 70570417 rows to 2009-04-08 12:52:22
Exported 70670417 rows to 2009-04-15 00:43:26
Exported 70770417 rows to 2009-04-20 19:08:57
Exported 70870417 rows to 2009-04-27 21:21:51
Exported 70970417 rows to 2009-05-05 14:01:37
Exported 71070417 rows to 2009-05-11 16:51:05
Exported 71170417 rows to 2009-05-

Exported 86358549 rows to 2012-03-06 16:21:29
Exported 86458549 rows to 2012-03-14 09:30:56
Exported 86558549 rows to 2012-03-19 15:36:56
Exported 86658549 rows to 2012-03-26 20:42:05
Exported 86758549 rows to 2012-04-01 12:20:06
Exported 86858549 rows to 2012-04-08 19:17:59
Exported 86958549 rows to 2012-04-14 15:38:12
Exported 87058549 rows to 2012-04-21 12:19:21
Exported 87158549 rows to 2012-04-27 10:53:44
Exported 87258549 rows to 2012-05-05 05:27:11
Exported 87358549 rows to 2012-05-10 07:25:38
Exported 87458549 rows to 2012-05-17 22:09:01
Exported 87558549 rows to 2012-05-24 14:43:49
Exported 87658549 rows to 2012-05-30 11:44:38
Exported 87758549 rows to 2012-06-06 21:12:39
Exported 87858549 rows to 2012-06-12 10:35:39
Exported 87958549 rows to 2012-06-19 21:17:45
Exported 88058549 rows to 2012-06-26 10:22:59
Exported 88158549 rows to 2012-07-03 00:34:15
Exported 88258549 rows to 2012-07-08 21:02:06
Exported 88358549 rows to 2012-07-15 17:57:56
Exported 88458549 rows to 2012-07-

Exported 103416669 rows to 2016-05-03 07:57:41
Exported 103516669 rows to 2016-05-07 20:00:50
Exported 103616669 rows to 2016-05-12 17:27:42
Exported 103716669 rows to 2016-05-18 22:18:27
Exported 103816669 rows to 2016-05-24 10:14:35
Exported 103916669 rows to 2016-05-29 13:05:09
Exported 104016669 rows to 2016-06-04 19:34:55
Exported 104116669 rows to 2016-06-09 20:14:02
Exported 104216669 rows to 2016-06-15 18:24:15
Exported 104316669 rows to 2016-06-20 14:43:25
Exported 104416669 rows to 2016-06-27 05:01:14
Exported 104516669 rows to 2016-07-01 22:04:52
Exported 104616669 rows to 2016-07-07 17:04:58
Exported 104716669 rows to 2016-07-12 01:30:20
Exported 104816669 rows to 2016-07-18 22:43:48
Exported 104916669 rows to 2016-07-23 20:28:11
Exported 105016669 rows to 2016-07-28 13:18:44
Exported 105116669 rows to 2016-08-04 20:15:49
Exported 105216669 rows to 2016-08-10 16:39:51
Exported 105316669 rows to 2016-08-15 22:16:42
Exported 105416669 rows to 2016-08-21 09:44:31
Exported 1055

Exported 120496680 rows to 2018-07-16 13:38:49
Exported 120596680 rows to 2018-07-20 08:58:14
Exported 120696680 rows to 2018-07-23 23:02:24
Exported 120796680 rows to 2018-07-28 09:00:30
Exported 120896680 rows to 2018-08-03 12:13:31
Exported 120996680 rows to 2018-08-07 16:43:52
Exported 121096680 rows to 2018-08-12 18:51:04
Exported 121196680 rows to 2018-08-17 22:38:33
Exported 121296680 rows to 2018-08-22 21:47:39
Exported 121396680 rows to 2018-08-27 07:50:18
Exported 121496680 rows to 2018-08-30 22:29:27
Exported 121596680 rows to 2018-09-04 10:14:09
Exported 121696680 rows to 2018-09-09 20:09:47
Exported 121796680 rows to 2018-09-13 19:17:29
Exported 121896680 rows to 2018-09-17 14:47:36
Exported 121996680 rows to 2018-09-22 11:20:41
Exported 122096680 rows to 2018-09-25 23:12:36
Exported 122196680 rows to 2018-10-01 09:52:38
Exported 122296680 rows to 2018-10-03 23:46:18
Exported 122396680 rows to 2018-10-09 23:48:05
Exported 122496680 rows to 2018-10-14 13:09:01
Exported 1225

In [9]:
!cd /mnt/disk100/persist_home/meawoppl/
!ls /mnt/disk100/persist_home/meawoppl/

export_test.h5	nb  reexport.h5


In [10]:
!du -sch *

392K	export_test.h5
12K	TLE Adventure.ipynb
8.0K	TLE_Compute_Timing.ipynb
4.0K	TLE_h5_build_index.ipynb
8.0K	TLE_precompute.ipynb
80K	TLE_to_pytables.ipynb
504K	total
